In [ ]:
! pip install -q "gdown==5.2.0" "datasets==2.20.0" "flash_attn==2.6.3" "timm==1.0.8" "einops==0.8.0" "transformers==4.44.0" pillow huggingface_hub

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import torch
import os
import pandas as pd
import numpy as np
import requests
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM, AutoConfig

In [ ]:
login(token=userdata.get('niru_hf_read'))

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
model = AutoModelForCausalLM.from_pretrained("nirusanan/Florence-2_FT_Lung-Cancer-detection", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("nirusanan/Florence-2_FT_Lung-Cancer-detection", trust_remote_code=True)

In [ ]:
prompt = "<DocVQA>" + "What is the type of lung cancer?"

url = "https://www.uab.edu/news/images/ct_scan.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<DocVQA>", image_size=(image.width, image.height))

print(parsed_answer)

{'<DocVQA>': 'Malignant cases'}
